In [ ]:
# install requirements
! pip install -r requirements.txt

In [1]:
import re
import pandas as pd
import math
import camelot
import dateutil

### The Notebook will show the flow for a single file and the code will be exported to a python script for more generalized solution

In [2]:
filename = 'data/canopy_technical_test_input.pdf'

The file has 1 table, detected using stream method of table detection

In [3]:
tables = camelot.read_pdf(filename, flavor='stream', pages='all')
tables

<TableList n=1>

Since the table detection is based of alignment, the left aligned lines at the top are also extracted, and need seperate cleaning

In [4]:
df = tables[0].df
df.head()

,0,1,2,3,4,5,6
0,31.03.2018 - 30.04.2018,,,,,,
1,Account SG1234567-01-01-JPY01 in JPY,,,,,,
2,Booking Details,,,,,,
3,Booking Date,Txn Date,Booking Text,Value Date,Debit,Credit,Balance
4,31.03.2018,31.03.2018,Initial Balance,,,,0.00


### Cleaning
Remove the first few rows that are not part of the table.  
Assuming each column should have a valid header, we use the first row that has call the columns non-empty as the header

In [5]:
class NoHeaderFound(ValueError): pass

def valid_header(row):
    """A row is valid is all the cells elements are neither nan nor empty string"""
    for j,elem in row.items():
        # if empty string or nan, skip. check for float before calling isnan to avoid TypeError
        if not elem or (isinstance(elem, float) and np.isnan(elem)):
            return False
    return True

def find_header_row(df):
    for i,row in df.iterrows():
        if valid_header(row): return i
    raise NoHeaderFound('No Rows satisfy the valid header condition')

h_row = find_header_row(df)
h_row

3

In [6]:
headers = list(df.iloc[h_row,:])
headers

['Booking Date',
 'Txn Date',
 'Booking Text',
 'Value Date',
 'Debit',
 'Credit',
 'Balance']

In [7]:
# strip the top part
df = df.iloc[h_row+1:,:]
# set the header
df.columns = headers

In [8]:
df.head()

,Booking Date,Txn Date,Booking Text,Value Date,Debit,Credit,Balance
4,31.03.2018,31.03.2018,Initial Balance,,,,0.00
5,01.04.2018,01.04.2018,VALUE DATED BALANCE BROUGHT FORWARD,01.04.2018,,"180,431,640.00","180,431,640.00"
6,06.04.2018,06.04.2018,INTEREST-FIXED TERM LOAN,06.04.2018,"472,500.00",,"179,959,140.00"
7,,,Contract No: 3001-AA18091ZN72C|Interest rate:,,,,
8,,,"0.810000%|Capital: 1,000,000,000.00|Period: 16...",,,,


### Merging the rows

#### Assumption
If a cell has a value, it will always start in first line, i.e., the cells are top aligned.

#### Caveat
There should atleast be one column that cannot have multi line values. This would mostly be a date or amount column.
This is required to determine the row seperation when all the columns have values. An empty value here is also acceptable but the value shouldn't span multiple lines.

This is generally not a problem for Financial Statements since each valid entry would have atleast one of date or amount which are single line values

### Conditions to find the row seperation
* If a value is missing in 1st line of a row is found, it signals start of a new row. eg. Credit value at index 5 above
* If all values are found in a line, determining the next line is tricky, in this case, we use the existance of single line value like Date or Amount to determine row separation  

##### Note
The above mentioned Conditions are just thoughts. As mentioned at the end of the Caveat, since Financial Statements have either of date or amount, we straight away use them to find the row separation

In [9]:
def is_date(candidate):
    """Check if a candidate can be parsed by the dateparser. Not necessarily accurate"""
    try:
        _ = dateutil.parser.parse(candidate)
        return True
    except dateutil.parser.ParserError:
        return False

def is_amount(candidate):
    """Is a valid amount.
    Doesn't support european way of writing the amount"""
    if re.match(r'^(\d+)(,(\d{2,3},)*\d{3})?(\.\d{2})?$', candidate): return True
    return False

### Find columns that have either date of amount only
These are used as row boundries

In [10]:
def get_single_val_cols(df):
    """Return column names that contain either of date or amount"""
    cols = []
    for col in df.columns:
        if all(df[col].apply(lambda elem: not elem or is_date(elem) or is_amount(elem))):
            cols.append(col)
    return cols
single_val_cols = get_single_val_cols(df)
single_val_cols

['Txn Date', 'Value Date', 'Debit', 'Credit', 'Balance']

In [11]:
# index if each valid entry starting row
starts = set()
for col in single_val_cols:
    starts = starts.union(df.index[df[col].astype(bool)])
# Add an extra index for merging trailing line if any
starts.add(max(df.index)+1)
starts = sorted(list(starts))
starts

[4, 5, 6, 10, 14, 16, 18, 24, 25, 29, 31, 33]

In [12]:
# Columns that can span multiple lines and need merging
multi_val_cols = [c for c in df.columns if c not in single_val_cols]
multi_val_cols

['Booking Date', 'Booking Text']

In [13]:
# We can see that `Remarks` is being picked Up in Booking Date column, making it not qualify as date.
# This garbage row will be trimmed just like the garbage rows at the top
df.tail()

,Booking Date,Txn Date,Booking Text,Value Date,Debit,Credit,Balance
28,,,"0.810000%|Capital: 500,000,000.00|Period: 27.0...",,,,
29,10.04.2018,10.04.2018,FOREX SPOT,10.04.2018,,"157,500.00","180,431,640.00"
30,,,EUR/JPY 131.1953,,,,
31,,,Carried forward,,,,"180,431,640.00"
32,Remarks,,,,,,


In [14]:
# Merge multi_val_cols between the row boundries
for i,j in zip(starts[:-1],starts[1:]):
    for m_col in multi_val_cols:
        df[m_col][i] = "\n".join([elem for elem in df[m_col].loc[i:j-1] if elem])

/home/dumbpy/applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Drop the rows multicol text of which has been merged above them

In [17]:
df = df.drop(set(df.index)-set(starts))
df = df.reset_index(drop=True)
df

,Booking Date,Txn Date,Booking Text,Value Date,Debit,Credit,Balance
0,31.03.2018,31.03.2018,Initial Balance,,,,0.00
1,01.04.2018,01.04.2018,VALUE DATED BALANCE BROUGHT FORWARD,01.04.2018,,"180,431,640.00","180,431,640.00"
2,06.04.2018,06.04.2018,INTEREST-FIXED TERM LOAN\nContract No: 3001-AA...,06.04.2018,"472,500.00",,"179,959,140.00"
3,06.04.2018,06.04.2018,INTEREST-FIXED TERM LOAN\nContract No: 3001-AA...,06.04.2018,"315,000.00",,"179,644,140.00"
4,06.04.2018,06.04.2018,FOREX SPOT\nEUR/JPY 130.7271,06.04.2018,,"472,500.00","180,116,640.00"
5,06.04.2018,06.04.2018,FOREX SPOT\nEUR/JPY 130.7021,06.04.2018,,"315,000.00","180,431,640.00"
6,09.04.2018,09.04.2018,INTEREST-FIXED TERM LOAN\nContract No: 3001-AA...,09.04.2018,"157,500.00",,"180,274,140.00"
7,09.04.2018\nE,09.04.2018,FOREX SPOT\nEUR/JPY 131.1407,09.04.2018,,"157,500.00","180,431,640.00"
8,10.04.2018,10.04.2018,INTEREST-FIXED TERM LOAN\nContract No: 3001-AA...,10.04.2018,"157,500.00",,"180,274,140.00"
9,10.04.2018,10.04.2018,FOREX SPOT\nEUR/JPY 131.1953,10.04.2018,,"157,500.00","180,431,640.00"
